In [ ]:
# repo_dir = "Repos"   # Set this to be where your github repos are located.
# %load_ext autoreload
# %autoreload 2

# # Update the load path so python can find modules for the model
# import sys
# from pathlib import Path
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
# sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))

In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
from deriva_ml.dataset_bag import DatasetBag

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml.deriva_ml_base import MLVocab as vc
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [2]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-12-05 09:24:42,381 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-12-05 09:24:42,382 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [3]:
cache_dir = Path.home() / '/data'
working_dir = Path.home() / '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)
pb = EA.catalog.getPathBuilder()

2024-12-05 09:24:44,343 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-12-05 09:24:44,344 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


# Configuration

In [4]:
# Add Workflow Vocab terms
EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="Dataset creation template",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_dataset.ipynb",
    workflow_type="Test Workflow"
)
# Configuration instance
config = ExecutionConfiguration(
    bdbags=['2-7K8W'], #2-7K8W
    models = ['2-4JR6'],
    execution=Execution(description="Sample Test Execution"),
    workflow=test_workflow,
    description="Our Test Workflow instance")
# Initialize execution
configuration_record = EA.initialize_execution(config)
execution_rid = configuration_record.execution_rid

2024-12-05 09:27:36,662 - INFO - Configuration validation successful!
2024-12-05 09:27:37,181 - INFO - Initializing downloader: GenericDownloader v1.7.4 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-12-05 09:27:37,183 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-12-05 09:27:37,184 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-12-05 09:27:37,185 - INFO - Validating credentials for host: dev.eye-ai.org
2024-12-05 09:27:37,198 - INFO - Creating bag directory: /tmp/tmp_y4cy368/Dataset_2-7K8W
2024-12-05 09:27:37,199 - INFO - Creating bag for directory /tmp/tmp_y4cy368/Dataset_2-7K8W
2024-12-05 09:27:37,200 - INFO - Creating data directory
2024-12-05 09:27:37,200 - INFO - Moving /tmp/tmp_y4cy368/Dataset_2-7K8W/tmpsfh0gi15 to data
2024-12-05 09:27

In [5]:
print(configuration_record)

caching_dir: /data
working_dir: /data/lizhiwei/EyeAI_working
execution_rid: 5-SHGC
workflow_rid: 5-SGDM
bag_paths: [PosixPath('/data/2-7K8W_dceecdb5229f5171b9f9530b685b80d6ced22a0b79028a1c6e8dce207b0ce9af/Dataset_2-7K8W')]
asset_paths: [PosixPath('/data/lizhiwei/EyeAI_working/5-SHGC/models/optic_disk_crop_model.hdf5')]
configuration: bdbags=['2-7K8W'] models=['2-4JR6'] workflow=Workflow(name='Dataset creation template', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_dataset.ipynb', workflow_type='Test Workflow', version=None, description=None) execution=Execution(description='Sample Test Execution') description='Our Test Workflow instance'


# Create DatasetBag

In [6]:
ds_bag = DatasetBag(configuration_record.bag_paths[0])

In [ ]:
training_rids = #slice the dataset and extract a list of subject rid

# Create dataset

In [ ]:
new_dataset_rid = EA.create_dataset(['LAC', 'Training'], description='A test dataset')

# Add subjects into the new dataset

In [ ]:
EA.add_dataset_members(dataset_rid=new_dataset_rid, members=training_rids)

# Add subdatasets to a dataset

In [ ]:
subsets = [] # A list of (sub)dataset rids
EA.add_dataset_members(dataset_rid=new_dataset_rid, members=subsets)

# Upload results

In [ ]:
# crete asset path
asset_type_name = ""
asset_path = configuration_record.execution_assets_path(asset_type_name)
# save assets to asset_path


# upload assets to catalog
EA.upload_execution(configuration_record, True)